Recommendation system which shows two lists, top products (max ten) with the highest discount and top ten products for which you save the most money (for a chosen area code).

In [10]:
import requests
import pandas as pd

# --- 1. API Token Setup ---
token = "SG_APIM_CM1M3GXGSA98V8PJ19BQDJPM238NHTTS5TVT7JM7Y2E2VEDBFQQ0"
headers = {
    "Authorization": f"Bearer {token}"
}

# --- 2. Load Danish ZIP codes ---
zip_codes = []
with open('DK.txt', 'r', encoding='utf-8') as f:
    for line in f:
        parts = line.strip().split('\t')
        if len(parts) > 1:
            zip_code = parts[1]
            zip_codes.append(zip_code)

# --- 3. User selects a ZIP code ---
print("\nAvailable ZIP codes (sample):", ", ".join(zip_codes[:20]), "...")
selected_zip = input("\nEnter a Danish ZIP code to search stores in: ").strip()

if selected_zip not in zip_codes:
    print("Invalid ZIP code. Exiting.")
    exit()
# --- 4. Fetch food waste data ---
def fetch_food_waste(zip_code):
    url = f"https://api.sallinggroup.com/v1/food-waste/?zip={zip_code}"
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error {response.status_code}: {response.text}")
        return None

food_waste_data = fetch_food_waste(selected_zip)

# --- 5. Build DataFrame ---
if food_waste_data:
    products = []
    for store in food_waste_data:
        store_name = store.get('store', {}).get('name', 'Unknown Store')
        for item in store.get('clearances', []):
            products.append({
                'description': item.get('product', {}).get('description', ''),
                'category': item.get('product', {}).get('categories', {}).get('en', 'Unknown'),
                'new_price': item.get('offer', {}).get('newPrice', None),
                'original_price': item.get('offer', {}).get('originalPrice', None),
                'percent_discount': item.get('offer', {}).get('percentDiscount', None),
                'stock': item.get('offer', {}).get('stock', None),
                'start_time': item.get('offer', {}).get('startTime', None),
                'end_time': item.get('offer', {}).get('endTime', None),
                'ean': item.get('product', {}).get('ean', None),
                'image_url': item.get('product', {}).get('image', None),
                'store': store_name
            })

    df = pd.DataFrame(products)

    # --- 6. Calculate money saved ---
    df['money_saved'] = df['original_price'] - df['new_price']

    # --- 7. Create recommendation lists ---
    top_percent_discount = df.sort_values(by='percent_discount', ascending=False).head(10)
    top_money_saved = df.sort_values(by='money_saved', ascending=False).head(10)

    # --- 8. ONLY Print the two top 10 lists ---
    print("\n🏆 Top Products by Highest Percentage Discount:")
    for idx, row in top_percent_discount.iterrows():
        print(f"- {row['description']} ({row['store']}): {row['percent_discount']:.1f}% off, now {row['new_price']} DKK (was {row['original_price']} DKK)")

    print("\n💰 Top Products by Most Money Saved:")
    for idx, row in top_money_saved.iterrows():
        print(f"- {row['description']} ({row['store']}): Saved {row['money_saved']:.2f} DKK, now {row['new_price']} DKK (was {row['original_price']} DKK)")

else:
    print("No data fetched.")


Available ZIP codes (sample): 1050, 1051, 1052, 1053, 1054, 1055, 1056, 1057, 1058, 1059, 1060, 1061, 1062, 1063, 1064, 1065, 1066, 1067, 1068, 1069 ...



Enter a Danish ZIP code to search stores in:  2400



🏆 Top Products by Highest Percentage Discount:
- HAMBURGERRYG PÅLÆKKER (Netto Tuborgvej): 68.7% off, now 5 DKK (was 15.95 DKK)
- DET GODE SOL RUGBRØD SCHULSTAD (Netto Lygten): 64.7% off, now 9 DKK (was 25.5 DKK)
- GREVEFEDT LAMPE (Netto Emdrupvej): 60.0% off, now 6 DKK (was 15.0 DKK)
- SURDEJSBOLLER ØGO (Netto Lygten): 57.8% off, now 8 DKK (was 18.95 DKK)
- SØDMÆLKSBRØD SCHULSTAD (Netto Lygten): 55.0% off, now 9 DKK (was 20.0 DKK)
- SKOVMAND SCHULSTAD (Netto Lygten): 54.7% off, now 6 DKK (was 13.25 DKK)
- BLINIS PREMIEUR (Netto Frederikssundsvej 52): 50.0% off, now 10 DKK (was 20.0 DKK)
- SCHWARZBROT GAMLE MØLLE (Netto Tomsgårdsvej): 50.0% off, now 4 DKK (was 8.0 DKK)
- TARTELETFYLD PREMIEUR (Netto Tomsgårdsvej): 50.0% off, now 25 DKK (was 50.0 DKK)
- MOUSSE M/ VANILJE PROTEIN LAB (Netto Lygten): 50.0% off, now 6 DKK (was 12.0 DKK)

💰 Top Products by Most Money Saved:
- SVINEMØRBRAD VELSMAG (Netto Tomsgårdsvej): Saved 44.95 DKK, now 82 DKK (was 126.95 DKK)
- TARTELETFYLD PREMIEUR (Net

Recommendation system for the highest discount and most money saved per category in a chosen area code. (Max three products per category).

In [8]:
import requests
import pandas as pd

# --- 1. API Token Setup ---
token = "SG_APIM_CM1M3GXGSA98V8PJ19BQDJPM238NHTTS5TVT7JM7Y2E2VEDBFQQ0"
headers = {
    "Authorization": f"Bearer {token}"
}

# --- 2. Load Danish ZIP codes ---
zip_codes = []
with open('DK.txt', 'r', encoding='utf-8') as f:
    for line in f:
        parts = line.strip().split('\t')
        if len(parts) > 1:
            zip_code = parts[1]
            zip_codes.append(zip_code)

# --- 3. User selects a ZIP code ---
print("\nAvailable ZIP codes (sample):", ", ".join(zip_codes[:20]), "...")
selected_zip = input("\nEnter a Danish ZIP code to search stores in: ").strip()

if selected_zip not in zip_codes:
    print("Invalid ZIP code. Exiting.")
    exit()

# --- 4. Fetch food waste data ---
def fetch_food_waste(zip_code):
    url = f"https://api.sallinggroup.com/v1/food-waste/?zip={zip_code}"
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error {response.status_code}: {response.text}")
        return None

food_waste_data = fetch_food_waste(selected_zip)

# --- 5. Build DataFrame ---
if food_waste_data:
    products = []
    for store in food_waste_data:
        store_name = store.get('store', {}).get('name', 'Unknown Store')
        for item in store.get('clearances', []):
            products.append({
                'description': item.get('product', {}).get('description', ''),
                'category': item.get('product', {}).get('categories', {}).get('en', 'Unknown'),
                'new_price': item.get('offer', {}).get('newPrice', None),
                'original_price': item.get('offer', {}).get('originalPrice', None),
                'percent_discount': item.get('offer', {}).get('percentDiscount', None),
                'stock': item.get('offer', {}).get('stock', None),
                'start_time': item.get('offer', {}).get('startTime', None),
                'end_time': item.get('offer', {}).get('endTime', None),
                'ean': item.get('product', {}).get('ean', None),
                'image_url': item.get('product', {}).get('image', None),
                'store': store_name
            })

    df = pd.DataFrame(products)

    # --- 6. Calculate money saved ---
    df['money_saved'] = df['original_price'] - df['new_price']

    # --- 7. Extract first category ---
    df['first_category'] = df['category'].apply(lambda x: x.split('>')[0] if isinstance(x, str) and '>' in x else x)

    # --- 8. For each category, print Top 3 products ---
    categories = df['first_category'].dropna().unique()

    for cat in categories:
        df_cat = df[df['first_category'] == cat]

        top_percent_discount = df_cat.sort_values(by='percent_discount', ascending=False).head(3)
        top_money_saved = df_cat.sort_values(by='money_saved', ascending=False).head(3)

        print(f"\n📚 Category: {cat}")

        print("\n🏆 Top products by Highest Percentage Discount:")
        for idx, row in top_percent_discount.iterrows():
            print(f"- {row['description']} ({row['store']}): {row['percent_discount']:.1f}% off, now {row['new_price']} DKK (was {row['original_price']} DKK)")

        print("\n💰 Top products by Most Money Saved:")
        for idx, row in top_money_saved.iterrows():
            print(f"- {row['description']} ({row['store']}): Saved {row['money_saved']:.2f} DKK, now {row['new_price']} DKK (was {row['original_price']} DKK)")

else:
    print("No data fetched.")


Available ZIP codes (sample): 1050, 1051, 1052, 1053, 1054, 1055, 1056, 1057, 1058, 1059, 1060, 1061, 1062, 1063, 1064, 1065, 1066, 1067, 1068, 1069 ...



Enter a Danish ZIP code to search stores in:  2400



📚 Category: Dairy And Cold Storage

🏆 Top products by Highest Percentage Discount:
- HAMBURGERRYG PÅLÆKKER (Netto Tuborgvej): 68.7% off, now 5 DKK (was 15.95 DKK)
- MOUSSE M/ VANILJE PROTEIN LAB (Netto Lygten): 50.0% off, now 6 DKK (was 12.0 DKK)
- TARTELETFYLD PREMIEUR (Netto Tomsgårdsvej): 50.0% off, now 25 DKK (was 50.0 DKK)

💰 Top products by Most Money Saved:
- TARTELETFYLD PREMIEUR (Netto Tomsgårdsvej): Saved 25.00 DKK, now 25 DKK (was 50.0 DKK)
- GRILLPØLSER ØGO (Netto Tuborgvej): Saved 15.95 DKK, now 24 DKK (was 39.95 DKK)
- ORGINAL SKIVER LEERDAMMER (Netto Tuborgvej): Saved 15.95 DKK, now 17 DKK (was 32.95 DKK)

📚 Category: Unknown

🏆 Top products by Highest Percentage Discount:
- GREVEFEDT LAMPE (Netto Emdrupvej): 60.0% off, now 6 DKK (was 15.0 DKK)
- BLINIS PREMIEUR (Netto Frederikssundsvej 52): 50.0% off, now 10 DKK (was 20.0 DKK)
- EMPANADAS OKSE LOS TAQUEROS (Netto Tomsgårdsvej): 48.6% off, now 18 DKK (was 35.0 DKK)

💰 Top products by Most Money Saved:
- GRILLBAKKE VELSM

Code which allows a user to pick an area code, and then to pick a store within that area code. Then they get a list of the products which are on sale within the store, recommending first the products which have the highest discount/allow you to save the most money.

In [11]:
import requests
import pandas as pd

# --- 1. API Token Setup ---
token = "SG_APIM_CM1M3GXGSA98V8PJ19BQDJPM238NHTTS5TVT7JM7Y2E2VEDBFQQ0"
headers = {
    "Authorization": f"Bearer {token}"
}

# --- 2. Load Danish ZIP codes ---
zip_codes = []
with open('DK.txt', 'r', encoding='utf-8') as f:
    for line in f:
        parts = line.strip().split('\t')
        if len(parts) > 1:
            zip_code = parts[1]
            zip_codes.append(zip_code)

# --- 3. User selects a ZIP code ---
print("\nAvailable ZIP codes (sample):", ", ".join(zip_codes[:20]), "...")
selected_zip = input("\nEnter a Danish ZIP code to search stores in: ").strip()

if selected_zip not in zip_codes:
    print("Invalid ZIP code. Exiting.")
    exit()

# --- 4. Fetch food waste data ---
def fetch_food_waste(zip_code):
    url = f"https://api.sallinggroup.com/v1/food-waste/?zip={zip_code}"
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error {response.status_code}: {response.text}")
        return None

food_waste_data = fetch_food_waste(selected_zip)

# --- 5. Let user choose a store ---
if food_waste_data:
    store_names = []
    store_lookup = {}
    for i, store in enumerate(food_waste_data):
        store_name = store.get('store', {}).get('name', f"Store #{i}")
        full_name = f"{store_name} - {store.get('store', {}).get('address', {}).get('street', '')}"
        store_names.append(full_name)
        store_lookup[str(i)] = store

    print("\nStores found in this area:")
    for i, name in enumerate(store_names):
        print(f"{i}: {name}")

    selected_index = input("\nSelect a store by number: ").strip()
    if selected_index not in store_lookup:
        print("Invalid selection. Exiting.")
        exit()

    selected_store = store_lookup[selected_index]

    # --- 6. Build DataFrame for that store ---
    store_name = selected_store.get('store', {}).get('name', 'Unknown Store')
    products = []
    for item in selected_store.get('clearances', []):
        products.append({
            'description': item.get('product', {}).get('description', ''),
            'category': item.get('product', {}).get('categories', {}).get('en', 'Unknown'),
            'new_price': item.get('offer', {}).get('newPrice', None),
            'original_price': item.get('offer', {}).get('originalPrice', None),
            'percent_discount': item.get('offer', {}).get('percentDiscount', None),
            'stock': item.get('offer', {}).get('stock', None),
            'start_time': item.get('offer', {}).get('startTime', None),
            'end_time': item.get('offer', {}).get('endTime', None),
            'ean': item.get('product', {}).get('ean', None),
            'image_url': item.get('product', {}).get('image', None)
        })

    df = pd.DataFrame(products)

    # --- 7. Calculate money saved ---
    df['money_saved'] = df['original_price'] - df['new_price']

    # --- 8. Create and print recommendation lists ---
    top_percent_discount = df.sort_values(by='percent_discount', ascending=False).head(10)
    top_money_saved = df.sort_values(by='money_saved', ascending=False).head(10)

    print(f"\n📍 Showing products for store: {store_name}")

    print("\n🏆 Top Products by Highest Percentage Discount:")
    for idx, row in top_percent_discount.iterrows():
        print(f"- {row['description']}: {row['percent_discount']:.1f}% off, now {row['new_price']} DKK (was {row['original_price']} DKK)")

    print("\n💰 Top Products by Most Money Saved:")
    for idx, row in top_money_saved.iterrows():
        print(f"- {row['description']}: Saved {row['money_saved']:.2f} DKK, now {row['new_price']} DKK (was {row['original_price']} DKK)")

else:
    print("No data fetched.")


Available ZIP codes (sample): 1050, 1051, 1052, 1053, 1054, 1055, 1056, 1057, 1058, 1059, 1060, 1061, 1062, 1063, 1064, 1065, 1066, 1067, 1068, 1069 ...



Enter a Danish ZIP code to search stores in:  2400



Stores found in this area:
0: Netto Emdrupvej - Emdrupvej 107
1: Netto Frederikssundsvej 52 - Frederikssundsvej 52 ST.
2: Netto Utterslevvej - Utterslevvej 11
3: Netto Tomsgårdsvej - Tomsgårdsvej 24
4: Netto Lygten - Lygten 53
5: Netto Stærevej - Stærevej 74
6: Netto Tuborgvej - Tuborgvej 239



Select a store by number:  2



📍 Showing products for store: Netto Utterslevvej

🏆 Top Products by Highest Percentage Discount:
- GROVHAK. LEVERPOSTEJ STRYHNS: 49.9% off, now 15 DKK (was 29.95 DKK)
- HAMMEL SPEGEPØLSE DELIKA: 49.9% off, now 10 DKK (was 19.95 DKK)
- SAN. KYL. TOMAT PÅLÆKKER: 49.8% off, now 8 DKK (was 15.95 DKK)
- KYL.BRYST M/URTER MINIMUM: 48.4% off, now 8 DKK (was 15.5 DKK)
- HVIDLØGSPATÉ LA CAMPAGNA: 46.7% off, now 10 DKK (was 18.75 DKK)
- FRIKADELLEPØLSE EGELYKKE: 39.8% off, now 9 DKK (was 14.95 DKK)
- ØKO SKUMMETMÆLK LØGISMOSE: 35.3% off, now 11 DKK (was 17.0 DKK)
- KARRY SALAT GRAASTEN: 25.0% off, now 9 DKK (was 12.0 DKK)
- ØKO YMER ARLA: 21.6% off, now 18 DKK (was 22.95 DKK)

💰 Top Products by Most Money Saved:
- GROVHAK. LEVERPOSTEJ STRYHNS: Saved 14.95 DKK, now 15 DKK (was 29.95 DKK)
- HAMMEL SPEGEPØLSE DELIKA: Saved 9.95 DKK, now 10 DKK (was 19.95 DKK)
- HVIDLØGSPATÉ LA CAMPAGNA: Saved 8.75 DKK, now 10 DKK (was 18.75 DKK)
- SAN. KYL. TOMAT PÅLÆKKER: Saved 7.95 DKK, now 8 DKK (was 15.95 DKK)